In [5]:
###############################################################################################################################################################################################
###### THIS CELL CONFIGURES THE NOTEBOOK, IMPORTS LIBRARIES AND DECLARE GLOBALS THAT ARE USED THROUGHOUT THIS NOTEBOOK. UPDATE ACCORDING DO YOUR CONFIGURATION + WHAT YOU WANT TO PLOT ########
###############################################################################################################################################################################################
import seaborn as sns                                                                                    
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import argparse
import os
from os import listdir
import copy
import pickle
import numpy as np
from scipy import stats

%matplotlib notebook

pd.set_option('display.max_columns', None)
pd.set_option('float_format', '{:f}'.format)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

rate = ''
exp_base_folder = os.path.join('/home/maxdml/experiments/', rate, '')
app_name = 'dmtr_http_srv'
client_name = 'rate_client'
stack_name = 'lwip'


##################### Traces description
# 1. CLT_PUSH_START - SENDING           Time between the scheduling of the request and its actual processing
# 2. CLT_PUSH_END - CLT_PUSH_START      Time to prepare the packet, send it to the NIC driver through rte_eth_tx_burst(), and free the dpdk mbuf
# 3. SRV_POP_START - CLT_PUSH_END       Time on the wire: item detected in the io queue's receive queue - client packet sent /!\ I think this can be negative if the server schedule's pop way before the client sends requests
# 4. SRV_POP_END - SRV_POP_START        Time to parse incoming packet + "waiting time" at the server's queue
# 5. NET_RECEIVE - SRV_POP_END          Time between message delivered to the application by dmtr_wait_any() and packet processed by the I/O queue
# 6. HTTP_DISPATCH - NET_RECEIVE        Time taken to select the HTTP recipient (either RR, or apply the filter, etc)
# 7. START_HTTP - HTTP_DISPATCH         Time spent in memory queue between network component and HTTP
# 8. END_HTTP - START_HTTP              Time spent performing HTTP processing
# 9. HTTP_DONE - END_HTTP               Time spent in memory queue between HTTP component and HTPP /!\ This include the "wait time" of dmtr_wait_any, as the same poll operates on both network sockets, and this memory queue
# 10. SRV_PUSH_START - HTTP_DONE        Time between the scheduling of the response and its actual processing
# 11. SRV_PUSH_END - SRV_PUSH_START     Time spent preparing the packet and sending it to the wire (identical to #2)
# 12. CLT_POP_START - SRV_PUSH_END      Time spent on the wire /!\ I think this can be negative as the client schedules the read as soon as it as sent the request
# 13. CLT_POP_END - CLT_POP_START       Time spent processing an incoming network packet (includes wait time) (identical to #4)
# 14. COMPLETED - CLT_POP_END           Time ellapsed between the reponse being delivered to the client by dmtr_wait_any(), and the response's being fully processed by the I/O queue


# We could add a POP_WAIT for the time between a pop running but waiting, and its actual execution.

In [69]:
exps = ['test']
exp_data = {}
for exp in exps:
    exp_data[exp] = pd.DataFrame()
    # First fetch server traces
    filename = exp +  '-traces-1' #FIXME add support for multiple network components
    serv_traces_f = os.path.join(exp_base_folder, exp, app_name, filename)
    server_traces = pd.read_csv(serv_traces_f, delimiter='\t')
#     print(server_traces)
    
    # Merge with demeter token traces
    filename_pattern = '-pop-traces'
    folderpath = os.path.join(exp_base_folder, exp, app_name)
    # Pop files
    pop_files = [os.path.join(folderpath, f) for f in listdir(folderpath) if os.path.isfile(os.path.join(folderpath, f)) and f.endswith(filename_pattern)]
    df = pd.read_csv(pop_files[0], delimiter='\t')
    server_df = pd.merge(server_traces, df, on='POP_TOKEN')
    server_df['SRV_POP_START'] = server_df[server_df.START == True].TIME
    server_df['SRV_POP_END'] = server_df[server_df.START == False].TIME
    server_df = server_df.groupby(['REQ_ID']).first().drop(['TIME', 'START', 'POP_TOKEN'], axis=1).reset_index() # get the first/last non-null value for each column within the group
    # Push
    filename_pattern = '-push-traces'
    push_files = [os.path.join(folderpath, f) for f in listdir(folderpath) if os.path.isfile(os.path.join(folderpath, f)) and f.endswith(filename_pattern)]
    df = pd.read_csv(push_files[0], delimiter='\t') 
    server_df = pd.merge(server_df, df, on='PUSH_TOKEN')
    server_df['SRV_PUSH_START'] = server_df[server_df.START == True].TIME
    server_df['SRV_PUSH_END'] = server_df[server_df.START == False].TIME
    server_df = server_df.groupby(['REQ_ID']).first().drop(['TIME', 'START', 'PUSH_TOKEN'], axis=1).reset_index() # get the first/last non-null value for each column within the group
        
    # Then idem for client traces
    filename = exp + '_traces'
    client_traces_f = os.path.join(exp_base_folder, exp, client_name, filename)
    client_traces = pd.read_csv(client_traces_f, delimiter='\t')
    
    # Merge with demeter token traces
    filename_pattern = '-pop-traces'
    folderpath = os.path.join(exp_base_folder, exp, client_name)
    # Pop files
    pop_files = [os.path.join(folderpath, f) for f in listdir(folderpath) if os.path.isfile(os.path.join(folderpath, f)) and f.endswith(filename_pattern)]
    df = pd.read_csv(pop_files[0], delimiter='\t')
    client_df = pd.merge(client_traces, df, on='POP_TOKEN')
    client_df['CLT_POP_START'] = client_df[client_df.START == True].TIME
    client_df['CLT_POP_END'] = client_df[client_df.START == False].TIME
    client_df = client_df.groupby(['REQ_ID']).first().drop(['TIME', 'START', 'POP_TOKEN'], axis=1).reset_index() # get the first/last non-null value for each column within the group
    # Push
    filename_pattern = '-push-traces'
    push_files = [os.path.join(folderpath, f) for f in listdir(folderpath) if os.path.isfile(os.path.join(folderpath, f)) and f.endswith(filename_pattern)]
    df = pd.read_csv(push_files[0], delimiter='\t') 
    client_df = pd.merge(client_df, df, on='PUSH_TOKEN')
    client_df['CLT_PUSH_START'] = client_df[client_df.START == True].TIME
    client_df['CLT_PUSH_END'] = client_df[client_df.START == False].TIME
    client_df = client_df.groupby(['REQ_ID']).first().drop(['TIME', 'START', 'PUSH_TOKEN'], axis=1).reset_index() # get the first/last non-null value for each column within the group
    
    cols = ['SENDING', 'CLT_PUSH_START', 'CLT_PUSH_END', 'SRV_POP_START', 'SRV_POP_END', 'NET_RECEIVE', 'HTTP_DISPATCH', 'START_HTTP', 'END_HTTP', 'HTTP_DONE', 'SRV_PUSH_START', 'SRV_PUSH_END', 'CLT_POP_START', 'CLT_POP_END', 'COMPLETED']
    full_df = pd.merge(server_df, client_df, on='REQ_ID').astype('int64')
    full_df = full_df[cols]
    full_df = full_df.diff(axis=1)
    full_df.drop(['SRV_POP_START', 'CLT_POP_START'], inplace=True, axis=1)
    full_df
    print(full_df)
#     import seaborn as sns
#     sns.set()
#     full_df.T.plot(kind='bar', stacked=True)
    